In [1]:
from datetime import datetime
from packaging import version
import os

In [2]:
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.9.0


In [3]:
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [5]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dataset mnist downloaded and prepared to C:\Users\camko\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [6]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(128)

In [7]:
ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [9]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])

Epoch 1/2
469/469 [==============================] - 21s 25ms/step - loss: 0.3586 - accuracy: 0.9012 - val_loss: 0.1936 - val_accuracy: 0.9457
Epoch 2/2
469/469 [==============================] - 11s 24ms/step - loss: 0.1661 - accuracy: 0.9526 - val_loss: 0.1380 - val_accuracy: 0.9605


In [11]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs